In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bson.json_util import dumps
from pymongo import MongoClient


#instantiate the CRUD module
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.A(children=[html.Img(style={'width': '200px', 'height': '200px'},
                              src='data:image/png;base64,{}'.format(encoded_image.decode()))], 
           href='https://www.snhu.edu/')),
    html.Center(html.B(html.H1('Gaspers SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        className='row',
        style={'display':'flex'},
        children=[
            dcc.RadioItems(id='radio-select',
            options=[
                {'label':'Water Rescue', 'value':'WR'},
                {'label':'Mountain or Wilderness', 'value':'MtW'},
                {'label':'Disaster or Individual Tracking', 'value':'DIT'},
                {'label':'Default', 'value':'Default'}],
            value='Default',
            labelStyle={'display':'inline-block'})
        ]
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        sort_action='native',
        filter_action='native',
        row_deletable=False,
        page_action='native',
        page_size=10,
        page_current=0,
    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row', style={'display':'flex'}, children=[
         html.Div(
             id='graph-id',
             className='col s12 m6',
             ),
         html.Div(
            id='map-id',
            className='col s12 m6',
            )
     ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

@app.callback(
    Output('datatable-id', 'data'),
    [Input('radio-select', 'value')])
def radioSelect(value):
    if (value == 'Default'):
        df = pd.DataFrame.from_records(shelter.read({}))
    elif (value == 'WR'):
        df = pd.DataFrame(list(shelter.read({'animal_type':'Dog',
                                             'breed':{'$in':['Labrador Retriever Mix',
                                                           'Chesapeake Bay Retriever',
                                                           'Newfoundland']},
                                            'sex_upon_outcome':'Intact Female',
                                            'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}})))
    elif (value == 'MtW'):
        df = pd.DataFrame(list(shelter.read({'animal_type':'Dog',
                                             'breed':{'$in':['German Shepard',
                                                           'Alaskan Malamute',
                                                           'Old English',
                                                            'Sheepdog',
                                                            'Siberian Husky',
                                                            'Rottweiler']},
                                            'sex_upon_outcome':'Intact Male',
                                            'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}})))
    elif (value == 'DIT'):
        df = pd.DataFrame(list(shelter.read({'animal_type':'Dog',
                                             'breed':{'$in':['Doberman Pinscher',
                                                           'German Shepard',
                                                           'Golden Retriever',
                                                            'Bloodhound',
                                                            'Rottweiler']},
                                            'sex_upon_outcome':'Intact Male',
                                            'age_upon_outcome_in_weeks':{'$gte':20, '$lte':300}})))
    return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#update the graph as the datatable updates
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', 'derived_viewport_data')])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(figure= px.pie(dff, values=dff.age_upon_outcome_in_weeks, 
                                 names=dff.breed, color_discrete_sequence=px.colors.sequential.RdBu))
    ]
    
#function to update the leaflet map as the datatable updates
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app